In [1]:
import os
from pathlib import Path


os.chdir(Path.cwd().parent)

In [2]:
import numpy as np

from src.rayleigh_quotient import rayleigh_quotient
from src.rayleigh_quotient_gradient import rayleigh_quotient_gradient, rayleigh_quotient_gradient_naive
from src.utils.linalg import generate_normalized_vector, generate_supersymmetric_tensor, normalize_vector
from src.utils.testing import timer

In [3]:
def test_rayleight_quotient_gradient_base(A: np.ndarray,
                                          X: np.ndarray,
                                          use_timer: bool = False,
                                          verbose: bool = False) -> tuple[int, int] | None:
    """
    Test Rayleigh quotient gradient computation with for loops and with numpy.

    :param A: supersymmetric tensor of shape (n x n x ... x n) of order k
    :param X: normalized vector of shape (n,)
    :param use_timer: whether to measure duration of computations
    :param verbose: whether to print more info

    :return: durations of naive and numpy approaches if |use_timer| is set
    """
    if verbose:
        print(f"testing Rayligh quotient gradient computation with parameters:\n"
              f"tensor a of shape {A.shape}\na = {A}\n"
              f"vector x = {X}")
    
    rqg_naive, duration_naive = timer(rayleigh_quotient_gradient_naive, A, X)
    rqg_numpy, duration_numpy = timer(rayleigh_quotient_gradient, A, X)

    assert np.linalg.norm(rqg_naive - rqg_numpy) < 0.001,\
           f"Following values should be equal:\n{rqg_naive=}\n{rqg_numpy=}"

    if use_timer:
        return duration_naive, duration_numpy

In [4]:
def test_rayleight_quotient_gradient_stress() -> None:
    n = 6
    k = 6
    for _ in range(10):
        a = generate_supersymmetric_tensor(n, k)
        x = generate_normalized_vector(n)
        test_rayleight_quotient_gradient_base(a, x)

def test_rayleight_quotient_gradient_stress_timer() -> None:
    n = 6
    k = 6
    durations_naive = []
    durations_numpy = []
    for _ in range(10):
        a = generate_supersymmetric_tensor(n, k)
        x = generate_normalized_vector(n)
        duration_naive, duration_numpy = test_rayleight_quotient_gradient_base(a, x, use_timer=True)
        durations_naive.append(duration_naive)
        durations_numpy.append(duration_numpy)

    print(f"naive estimation duration: {np.mean(durations_naive)}")
    print(f"numpy estimation duration: {np.mean(durations_numpy)}")


In [5]:
test_rayleight_quotient_gradient_stress()
test_rayleight_quotient_gradient_stress_timer()

naive estimation duration: 0.5364293575286865
numpy estimation duration: 0.0001625537872314453


In [6]:
def test_gradient_approximation(n: int, k: int, eps: float = 1e-8) -> None:
    print(f"test gradient approximation with {n=}, {k=}")
    errors = []
    for _ in range(10):
        A = generate_supersymmetric_tensor(n, k)
        for __ in range(10):
            x = generate_normalized_vector(n)
            grad = rayleigh_quotient_gradient(A, x)
            for ___ in range(10):
                t = generate_normalized_vector(n)
                rq1 = rayleigh_quotient(A, normalize_vector(x + eps * t))
                rq2 = rayleigh_quotient(A, normalize_vector(x - eps * t))
                grad_approx = (rq1 - rq2) / (2 * eps)
                error = np.linalg.norm(grad @ t - grad_approx)
                errors.append(error)

    print(f"max error = {max(errors)}")
    print(f"mean error = {np.mean(errors)}")

In [7]:
test_gradient_approximation(n=2, k=2)

test gradient approximation with n=2, k=2
max error = 4.8329193536034154e-08
mean error = 8.597349251807107e-09


In [8]:
test_gradient_approximation(n=3, k=3)

test gradient approximation with n=3, k=3
max error = 4.5199843090593106e-07
mean error = 7.573425512527598e-08


In [9]:
test_gradient_approximation(n=6, k=6, eps=1e-6)

test gradient approximation with n=6, k=6
max error = 1.9186963982065208e-05
mean error = 3.5854217794621945e-06
